<a href="https://colab.research.google.com/github/tonybuii2003/AITourGuide/blob/main/tourguidegpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [34]:
from IPython.display import Markdown, display

In [2]:
pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
from openai import AsyncOpenAI
import asyncio

In [12]:
import os
from dotenv import load_dotenv

In [3]:
data = pd.read_csv('/Users/flying-dragon03/Documents/projects/TourGuideApp/openaccess/MetObjects.csv')

/var/folders/s5/pg7q47_11hbf9hmbwtn2z1dc0000gn/T/ipykernel_36902/1170850707.py:1: DtypeWarning: Columns (5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/flying-dragon03/Documents/projects/TourGuideApp/openaccess/MetObjects.csv')


In [6]:
print(data.columns.values)

['Object Number' 'Is Highlight' 'Is Timeline Work' 'Is Public Domain'
 'Object ID' 'Gallery Number' 'Department' 'AccessionYear' 'Object Name'
 'Title' 'Culture' 'Period' 'Dynasty' 'Reign' 'Portfolio' 'Constituent ID'
 'Artist Role' 'Artist Prefix' 'Artist Display Name' 'Artist Display Bio'
 'Artist Suffix' 'Artist Alpha Sort' 'Artist Nationality'
 'Artist Begin Date' 'Artist End Date' 'Artist Gender' 'Artist ULAN URL'
 'Artist Wikidata URL' 'Object Date' 'Object Begin Date' 'Object End Date'
 'Medium' 'Dimensions' 'Credit Line' 'Geography Type' 'City' 'State'
 'County' 'Country' 'Region' 'Subregion' 'Locale' 'Locus' 'Excavation'
 'River' 'Classification' 'Rights and Reproduction' 'Link Resource'
 'Object Wikidata URL' 'Metadata Date' 'Repository' 'Tags' 'Tags AAT URL'
 'Tags Wikidata URL']


In [7]:
relevant_columns = [
    'Title', 'Culture', 'Period', 'Dynasty', 'Reign',
    'Artist Display Name', 'Artist Nationality', 'Artist Display Bio',
    'Object Date', 'Medium', 'Dimensions', 'Credit Line',
    'City', 'Country', 'Region', 'Classification', 'Tags'
]

# Fill NaN values with empty strings to avoid errors during concatenation
data[relevant_columns] = data[relevant_columns].fillna('')

# Create a combined_text column with prioritized information
data['combined_text'] = data.apply(
    lambda row: ' | '.join([f"{col}: {row[col]}" for col in relevant_columns if row[col]]),
    axis=1
)

In [8]:
# Step 1: Retriever - TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['combined_text'])

In [31]:
load_dotenv()
api_key = os.getenv('OPENAI_API')
if api_key is None:
    raise ValueError("The 'OPENAI_API' key is missing from the environment variables.")
client = AsyncOpenAI(api_key=api_key)

In [32]:
async def rag_query(user_query):
    query_vector = vectorizer.transform([user_query])

    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()

    top_doc_index = similarity_scores.argmax()
    retrieved_context = data.iloc[top_doc_index]['combined_text']
    prompt = f"Context: {retrieved_context}\n\nQuestion: {user_query}\nAnswer:"
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert tour guide at The Metropolitan Museum of Art (The Met) in New York City. Your role is to provide engaging, accurate, and detailed information about the museum's exhibits, history, and artworks. Answer questions with enthusiasm, clarity, and a friendly tone, making the information accessible to all ages. Highlight interesting facts, cultural significance, and historical context when relevant. Feel free to recommend must-see exhibits, hidden gems, and tips for enjoying the museum experience. If the visitor asks in a specific language, respond in the same language fluently while maintaining the same level of detail and professionalism. Always adapt your tone to be culturally appropriate and respectful. If you don’t have enough information, respond honestly and encourage the visitor to explore further at The Met."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content



In [36]:
# Example Query
query = "kể tôi về 3 bức tranh có giá trị lịch sử lâu đời nhất ở Met"

try:
    import nest_asyncio
    nest_asyncio.apply()
    response = await rag_query(query)
except RuntimeError:
    response = asyncio.run(rag_query(query))

display(Markdown(response))


Dĩ nhiên! Tại The Met, có rất nhiều tác phẩm nghệ thuật có giá trị lịch sử lâu đời, và dưới đây là ba bức tranh nổi bật mà bạn có thể quan tâm:

1. **"Madonna and Child" của Duccio di Buoninsegna**: Được tạo ra vào khoảng năm 1300, bức tranh này là một trong những tác phẩm quan trọng của nghệ sĩ người Ý Duccio, nổi tiếng với phong cách hội họa Gothic. Bức tranh này không chỉ thể hiện kỹ thuật đỉnh cao thời Trung Cổ mà còn là một trong những tác phẩm nổi bật nhất trong bộ sưu tập nghệ thuật châu Âu của The Met.

2. **"The Harvesters" của Pieter Bruegel the Elder**: Được vẽ vào năm 1565, đây là một trong những kiệt tác của nghệ sĩ người Hà Lan Pieter Bruegel the Elder. Tác phẩm này là một phần của loạt tranh về các mùa trong năm và miêu tả cảnh nông thôn với chi tiết tỉ mỉ và màu sắc tươi sáng, thể hiện một góc nhìn độc đáo về đời sống nông dân vào thế kỷ 16.

3. **"The Death of Socrates" của Jacques-Louis David**: Được sáng tác vào năm 1787, bức tranh theo trường phái Tân cổ điển này miêu tả khoảnh khắc cuối cùng trong cuộc đời của triết gia Socrates. Jacques-Louis David, một trong những họa sĩ nổi tiếng nhất của thời kỳ này, đã sử dụng ánh sáng và bố cục một cách tài tình để truyền tải thông điệp về triết lý và sự hy sinh.

Hy vọng những thông tin này sẽ giúp bạn có một chuyến tham quan thú vị tại The Met! Nếu bạn cần thêm thông tin về bất kỳ tác phẩm nào, đừng ngần ngại yêu cầu.